In [ ]:
# Import required libraries
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
import os
from pycocotools.coco import COCO
import requests
from tqdm import tqdm
import json
import cv2
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")


In [ ]:
# Memory optimization utilities for Kaggle
import gc
import torch

def clear_memory():
    """Clear GPU and system memory"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print("✓ Memory cleared")

def print_memory_usage():
    """Print current GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    
print("✓ Memory utilities loaded")

## Configuration

Set your dataset generation parameters here:

### ⏱️ **Time Estimation for Kaggle (12hr limit):**
- **Qwen-VL-Chat inference**: ~3-5 seconds per image (with short captions)
- **5,000 train images**: ~4-7 hours
- **500 val images**: ~25-42 minutes
- **Total estimated time**: ~4.5-8 hours (fits within 12hr limit!)

**Optimizations to speed up:**
- Shorter captions (70 tokens max) = faster generation
- Batch processing where possible
- Skip failed downloads quickly
- GPU acceleration (T4/P100)

In [ ]:
# Dataset Generation Configuration
class Config:
    # Get notebook directory for absolute paths
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath('__file__')) if '__file__' in dir() else os.getcwd()
    
    # Paths (using absolute paths to avoid directory issues)
    COCO_ROOT = os.path.join(NOTEBOOK_DIR, 'coco_data')
    OUTPUT_DIR = os.path.join(NOTEBOOK_DIR, 'dataset_output')
    
    # Dataset sizes
    TARGET_TRAIN = 5000  # Training images
    TARGET_VAL = 500     # Validation images
    
    # Quality filtering thresholds
    MIN_KEYPOINTS = 10   # Minimum visible keypoints (out of 17)
    MIN_PERSON_AREA = 5000  # Minimum person area in pixels²
    
    # Caption generation
    MAX_CAPTION_TOKENS = 70  # CLIP limit is 77, we use 70 for safety
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Model - Using Qwen2-VL-2B (proven to work, no OOM)
    MODEL_NAME = "Qwen/Qwen2-VL-2B-Instruct"
    MODEL_TYPE = "qwen2-vl"

config = Config()

print("="*80)
print("CONFIGURATION")
print("="*80)
print(f"Working Directory: {config.NOTEBOOK_DIR}")
print(f"COCO Root: {config.COCO_ROOT}")
print(f"Output Directory: {config.OUTPUT_DIR}")
print(f"Target Train Images: {config.TARGET_TRAIN}")
print(f"Target Val Images: {config.TARGET_VAL}")
print(f"Quality Thresholds:")
print(f"  - Min Keypoints: {config.MIN_KEYPOINTS}/17")
print(f"  - Min Person Area: {config.MIN_PERSON_AREA}px²")
print(f"Caption Settings:")
print(f"  - Max Tokens: {config.MAX_CAPTION_TOKENS}")
print(f"  - Device: {config.DEVICE}")
print(f"  - Model: {config.MODEL_NAME}")
print(f"  - Model Type: {config.MODEL_TYPE}")
print(f"\n✓ Using Qwen2-VL-2B (proven working, no OOM issues)")
print(f"  - Smaller than Qwen-VL-Chat")
print(f"  - Disk cache system for captions")
print(f"  - Can unload model after caption generation")
print("="*80)

# Create directories
os.makedirs(config.COCO_ROOT, exist_ok=True)
os.makedirs(config.OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(config.COCO_ROOT, 'annotations'), exist_ok=True)
os.makedirs(os.path.join(config.COCO_ROOT, 'train2017'), exist_ok=True)
os.makedirs(os.path.join(config.COCO_ROOT, 'val2017'), exist_ok=True)

print(f"\n✓ Directories created/verified:")
print(f"  - {config.COCO_ROOT}")
print(f"  - {config.OUTPUT_DIR}")


## Step 1: Load COCO Annotations

Download and load COCO 2017 annotations for person keypoints.

In [ ]:
# Load COCO annotations
ann_dir = os.path.join(config.COCO_ROOT, 'annotations')
train_ann_file = os.path.join(ann_dir, 'person_keypoints_train2017.json')
val_ann_file = os.path.join(ann_dir, 'person_keypoints_val2017.json')

# Download annotations if they don't exist
if not os.path.exists(train_ann_file) or not os.path.exists(val_ann_file):
    print("⚠️  COCO annotations not found! Downloading...")
    
    import zipfile
    import urllib.request
    
    ann_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
    zip_path = os.path.join(config.COCO_ROOT, 'annotations_trainval2017.zip')
    
    print(f"Downloading from: {ann_url}")
    print("This may take a few minutes (~252 MB)...")
    
    # Download with progress
    urllib.request.urlretrieve(ann_url, zip_path)
    print("✓ Download complete! Extracting...")
    
    # Extract
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(config.COCO_ROOT)
    
    # Cleanup zip file
    os.remove(zip_path)
    print("✓ Extraction complete!")

# Now load COCO annotations
print("\nLoading COCO annotations...")
coco_train = COCO(train_ann_file)
coco_val = COCO(val_ann_file)
print("✓ COCO annotations loaded successfully!")
print(f"  - Train images: {len(coco_train.getImgIds())}")
print(f"  - Val images: {len(coco_val.getImgIds())}")

## Step 2: Filter High-Quality Pose Images

Filter COCO dataset for images with high-quality pose annotations.

In [ ]:
def filter_high_quality_images(coco, split='train', min_keypoints=10, min_area=5000, max_images=None):
    """
    Filter COCO dataset for high-quality pose images
    
    Args:
        coco: COCO API instance
        split: 'train' or 'val'
        min_keypoints: Minimum visible keypoints (out of 17)
        min_area: Minimum person area in pixels
        max_images: Maximum number of images to return
    
    Returns:
        list: Filtered image IDs with quality scores
    """
    print(f"\n{'='*80}")
    print(f"FILTERING {split.upper()} IMAGES FOR HIGH-QUALITY POSES")
    print(f"{'='*80}")
    print(f"Criteria:")
    print(f"  - Minimum keypoints: {min_keypoints}/17")
    print(f"  - Minimum person area: {min_area}px²")
    print(f"  - No crowd annotations")
    print(f"{'='*80}\n")
    
    cat_ids = coco.getCatIds(catNms=['person'])
    all_img_ids = coco.getImgIds(catIds=cat_ids)
    
    quality_images = []
    
    for img_id in tqdm(all_img_ids, desc=f"Filtering {split} images"):
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_ids, iscrowd=False)
        anns = coco.loadAnns(ann_ids)
        
        best_quality_score = 0
        best_keypoint_count = 0
        
        for ann in anns:
            if 'keypoints' not in ann:
                continue
            
            num_keypoints = ann.get('num_keypoints', 0)
            person_area = ann.get('area', 0)
            
            # Quality filtering
            if (num_keypoints >= min_keypoints and 
                person_area >= min_area and 
                ann.get('iscrowd', 0) == 0):
                
                # Quality score: combine keypoints and area
                quality_score = num_keypoints * 1.0 + (person_area / 10000) * 0.5
                
                if quality_score > best_quality_score:
                    best_quality_score = quality_score
                    best_keypoint_count = num_keypoints
        
        if best_quality_score > 0:
            quality_images.append({
                'image_id': img_id,
                'quality_score': best_quality_score,
                'keypoints': best_keypoint_count
            })
    
    # Sort by quality score (best first)
    quality_images.sort(key=lambda x: x['quality_score'], reverse=True)
    
    # Limit to max_images if specified
    if max_images and len(quality_images) > max_images:
        quality_images = quality_images[:max_images]
    
    print(f"\n✓ Filtered {len(quality_images)} high-quality images from {len(all_img_ids)} total")
    print(f"  - Average keypoints: {np.mean([img['keypoints'] for img in quality_images]):.1f}/17")
    print(f"  - Quality range: {quality_images[-1]['quality_score']:.2f} to {quality_images[0]['quality_score']:.2f}")
    
    return quality_images

# Check if COCO annotations are loaded
if 'coco_train' not in globals() or 'coco_val' not in globals():
    print("⚠️  ERROR: COCO annotations not loaded!")
    print("Please run Cell 5 (Step 1: Load COCO Annotations) first.")
    raise NameError("coco_train and coco_val are not defined. Run the previous cells in order.")

# Filter train and val images
train_images = filter_high_quality_images(
    coco_train, 
    split='train',
    min_keypoints=config.MIN_KEYPOINTS,
    min_area=config.MIN_PERSON_AREA,
    max_images=config.TARGET_TRAIN
)

val_images = filter_high_quality_images(
    coco_val, 
    split='val',
    min_keypoints=config.MIN_KEYPOINTS,
    min_area=config.MIN_PERSON_AREA,
    max_images=config.TARGET_VAL
)

print(f"\n{'='*80}")
print("FILTERING COMPLETE")
print(f"{'='*80}")
print(f"✓ Train: {len(train_images)} images selected")
print(f"✓ Val: {len(val_images)} images selected")
print(f"{'='*80}\n")

## Step 3: Initialize Caption Generator

Load Qwen-VL-Chat model for generating CLIP-compatible captions.

In [ ]:
# Install required dependencies for Qwen2-VL-2B
import subprocess
import sys
import gc

print("Installing dependencies for Qwen2-VL-2B...")

# First, clear any existing GPU memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✓ GPU memory cleared")

# Install compatible versions
packages = [
    'transformers>=4.45.0',
    'accelerate',
    'qwen-vl-utils',
    'pillow',
    'torchvision'
]

for package in packages:
    print(f"Installing {package}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("\n✓ All dependencies installed!")


In [ ]:
# GPU Memory Status Check
import gc

print("="*80)
print("GPU MEMORY STATUS")
print("="*80)

# Force garbage collection and clear cache
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
    # Get GPU memory info
    props = torch.cuda.get_device_properties(0)
    print(f"GPU: {props.name}")
    print(f"Total GPU Memory: {props.total_memory / 1e9:.2f} GB")
    
    # Get current memory usage
    current_allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    total = props.total_memory / 1e9
    free = total - (reserved)
    
    print(f"\nCurrent Memory Usage:")
    print(f"  - Allocated: {current_allocated:.2f} GB")
    print(f"  - Reserved: {reserved:.2f} GB")
    print(f"  - Free: {free:.2f} GB")
    
    if current_allocated > 10:
        print(f"\n⚠️  WARNING: High GPU memory usage!")
        print("Clearing unneeded variables...")
        # Try to identify and clear large objects
        if 'train_images' in globals() or 'val_images' in globals():
            print("  - Keeping image lists (needed for caption generation)")
        if 'coco_train' in globals() or 'coco_val' in globals():
            print("  - Keeping COCO objects (needed for image info)")
else:
    print("⚠️  CUDA not available!")

print("="*80 + "\n")


In [ ]:
class CaptionGenerator:
    """Ultra-optimized caption generator for Kaggle - minimal memory footprint"""
    
    def __init__(self, model_name="Qwen/Qwen2-VL-2B-Instruct"):
        """Initialize with aggressive memory management"""
        import gc
        from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
        
        print(f"Loading {model_name}...")
        
        # Pre-load cleanup
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        
        # Load processor
        self.processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)
        
        # Load model in fp16 for memory efficiency
        self.model = Qwen2VLForConditionalGeneration.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map="auto",
        ).eval()
        
        print(f"✓ Model loaded successfully")
        
        # Post-load cleanup
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
    def generate_caption(self, image_path):
        """Generate caption with minimal memory usage"""
        import gc
        from PIL import Image as PILImage
        
        try:
            # Pre-process cleanup
            gc.collect()
            torch.cuda.empty_cache()
            
            # Load and resize image (critical for memory)
            image = PILImage.open(image_path).convert('RGB')
            
            # Aggressive downsizing - max 768 pixels
            if max(image.size) > 768:
                ratio = 768 / max(image.size)
                new_size = (int(image.width * ratio), int(image.height * ratio))
                image = image.resize(new_size, PILImage.Resampling.LANCZOS)
            
            # Create prompt
            conversation = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": "Describe this image for image generation. Include: main subjects, their positions, colors, lighting, mood, background, and style. Be specific and vivid."}
                    ]
                }
            ]
            
            # Process with minimal overhead
            text_prompt = self.processor.apply_chat_template(
                conversation, tokenize=False, add_generation_prompt=True
            )
            
            inputs = self.processor(
                text=text_prompt,
                images=[image],
                return_tensors="pt",
                padding=True
            )
            
            # Move to device
            inputs = {k: v.to(self.model.device) if isinstance(v, torch.Tensor) else v 
                     for k, v in inputs.items()}
            
            # Generate with minimal parameters
            with torch.no_grad():
                output_ids = self.model.generate(
                    **inputs,
                    max_new_tokens=60,
                    do_sample=False
                )
            
            # Decode
            caption = self.processor.decode(output_ids[0], skip_special_tokens=True)
            
            # Clean caption
            caption = caption.strip()
            
            # Remove common prefixes
            for prefix in ['assistant:', '<|assistant|>', 'sure ', 'here ', 'the image']:
                if caption.lower().startswith(prefix):
                    caption = caption[len(prefix):].strip()
            
            # Enforce 70-token limit (hard cutoff)
            words = caption.split()
            if len(words) > 70:
                caption = ' '.join(words[:70])
            
            # Ensure proper ending
            if caption and not caption.endswith(('.', '!', '?')):
                caption += '.'
            
            # Immediate cleanup
            del image, inputs, output_ids, text_prompt, conversation
            gc.collect()
            torch.cuda.empty_cache()
            
            return caption if len(caption) > 5 else "A person in a scene."
            
        except Exception as e:
            print(f"⚠️  Error: {str(e)[:40]}")
            torch.cuda.empty_cache()
            gc.collect()
            return "A person in a scene."
    
    def cleanup(self):
        """Free all GPU memory"""
        import gc
        del self.model
        del self.processor
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print("✓ Model cleaned up")

In [ ]:
# Initialize Caption Generator
print("="*80)
print("INITIALIZING CAPTION GENERATOR")
print("="*80)
print("Loading Qwen2-VL-2B model (this will take 2-3 minutes)...")
print()

caption_gen = CaptionGenerator(model_name=config.MODEL_NAME)

print()
print("="*80)
print("✓ CAPTION GENERATOR READY!")
print("="*80)
print()
print_memory_usage()

## Step 4: Generate Training Captions

Download training images and generate detailed captions.

In [ ]:
# Generate Training Captions - Ultra-optimized
import gc
import requests

if 'caption_gen' not in globals():
    raise NameError("Run Cell 13 (Initialize Caption Generator) first!")

train_captions = {}
img_dir_train = os.path.join(config.COCO_ROOT, 'train2017')

print(f"\n{'='*60}")
print(f"GENERATING {len(train_images)} TRAINING CAPTIONS")
print(f"{'='*60}\n")

success = 0
failed = 0

for img_data in tqdm(train_images, desc="Train"):
    img_id = img_data['image_id']
    img_info = coco_train.loadImgs(img_id)[0]
    filename = img_info['file_name']
    img_path = os.path.join(img_dir_train, filename)
    
    # Download if missing
    if not os.path.exists(img_path):
        try:
            r = requests.get(img_info['coco_url'], timeout=10)
            r.raise_for_status()
            os.makedirs(os.path.dirname(img_path), exist_ok=True)
            with open(img_path, 'wb') as f:
                f.write(r.content)
        except:
            failed += 1
            continue
    
    # Generate caption
    try:
        caption = caption_gen.generate_caption(img_path)
        if caption and len(caption) > 3:
            train_captions[filename] = caption
            success += 1
        else:
            failed += 1
    except:
        failed += 1
    
    # Periodic cleanup
    if (success + failed) % 100 == 0:
        gc.collect()
        torch.cuda.empty_cache()

print(f"\n✓ Generated: {success}")
print(f"✗ Failed: {failed}")
print(f"{'='*60}\n")

## Step 5: Generate Validation Captions

Download validation images and generate detailed captions.

In [ ]:
# Generate Validation Captions - Ultra-optimized
import gc
import requests

if 'caption_gen' not in globals():
    raise NameError("Run Cell 13 (Initialize Caption Generator) first!")

val_captions = {}
img_dir_val = os.path.join(config.COCO_ROOT, 'val2017')

print(f"\n{'='*60}")
print(f"GENERATING {len(val_images)} VALIDATION CAPTIONS")
print(f"{'='*60}\n")

success = 0
failed = 0

for img_data in tqdm(val_images, desc="Val"):
    img_id = img_data['image_id']
    img_info = coco_val.loadImgs(img_id)[0]
    filename = img_info['file_name']
    img_path = os.path.join(img_dir_val, filename)
    
    # Download if missing
    if not os.path.exists(img_path):
        try:
            r = requests.get(img_info['coco_url'], timeout=10)
            r.raise_for_status()
            os.makedirs(os.path.dirname(img_path), exist_ok=True)
            with open(img_path, 'wb') as f:
                f.write(r.content)
        except:
            failed += 1
            continue
    
    # Generate caption
    try:
        caption = caption_gen.generate_caption(img_path)
        if caption and len(caption) > 3:
            val_captions[filename] = caption
            success += 1
        else:
            failed += 1
    except:
        failed += 1
    
    # Periodic cleanup
    if (success + failed) % 50 == 0:
        gc.collect()
        torch.cuda.empty_cache()

print(f"\n✓ Generated: {success}")
print(f"✗ Failed: {failed}")
print(f"{'='*60}\n")

## Step 6: Save Caption Files

Save the generated captions to JSON files.

In [ ]:
# Save Dataset & Cleanup
import json
import gc

# Cleanup model
if 'caption_gen' in globals():
    caption_gen.cleanup()

# Save captions
train_out = os.path.join(config.OUTPUT_DIR, 'train_captions.json')
val_out = os.path.join(config.OUTPUT_DIR, 'val_captions.json')

with open(train_out, 'w') as f:
    json.dump(train_captions, f, indent=2)

with open(val_out, 'w') as f:
    json.dump(val_captions, f, indent=2)

print(f"\n{'='*60}")
print("DATASET COMPLETE!")
print(f"{'='*60}")
print(f"Train: {len(train_captions)} captions → {train_out}")
print(f"Val: {len(val_captions)} captions → {val_out}")
print(f"\nStats:")
print(f"  Total: {len(train_captions) + len(val_captions)} images")
if train_captions:
    avg_train = sum(len(c.split()) for c in train_captions.values()) / len(train_captions)
    print(f"  Avg caption (train): {avg_train:.1f} tokens")
if val_captions:
    avg_val = sum(len(c.split()) for c in val_captions.values()) / len(val_captions)
    print(f"  Avg caption (val): {avg_val:.1f} tokens")
print(f"{'='*60}\n")

## Step 7: Verify Dataset Quality

Check a few samples to verify caption quality and token counts.

In [ ]:
# Verify Dataset (Quick Check Only)
print("\n" + "="*60)
print("SAMPLE CAPTIONS")
print("="*60)

if train_captions:
    for i, (fname, cap) in enumerate(list(train_captions.items())[:2]):
        print(f"\nTrain[{i}]: {fname}")
        print(f"  Caption: {cap[:100]}...")
        print(f"  Tokens: {len(cap.split())}")

if val_captions:
    for i, (fname, cap) in enumerate(list(val_captions.items())[:2]):
        print(f"\nVal[{i}]: {fname}")
        print(f"  Caption: {cap[:100]}...")
        print(f"  Tokens: {len(cap.split())}")

print("\n✅ Ready for ControlNet training!")
print("="*60)